## TODO
- [ ] try masking and erode to remove bands
- [x] write output as a COG
- [ ] crop to AOI

In [9]:
%load_ext autoreload
%autoreload 2
# import pyroSAR
from pyroSAR.snap.auxil import Workflow, gpt, groupbyWorkers, parse_recipe, parse_node
from s1pro.auxils import get_burst_geometry
import shapely
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio as rio
import numpy as np
from rasterio import merge
from rasterio.io import MemoryFile
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
from scipy.ndimage import binary_erosion

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
infiles = ['/data/S1/S1A_IW_SLC__1SDV_20230826T180527_20230826T180554_050050_0605B2_E768.zip',
           '/data/S1/S1A_IW_SLC__1SDV_20230814T180526_20230814T180553_049875_05FFB4_7D92.zip']
outdir = '/data/res/'
aoi_path = '/data/aoi_bretagne.geojson'
# graph_path = '../graph/TOPSAR Coreg Interferogram IW2 to GeoTiff.xml'
# graph_path = '../graph/TOPSAR Coreg Coherence VV IW2 to GeoTiff.xml'
graph_path = '../graph/TOPSAR_coh_geocode_IW_to_geotiff.xml'

In [11]:
# let's find what iw and bursts intersect AOI
df_geo = get_burst_geometry(infiles[0], 
                   target_subswaths=['IW1', 'IW2','IW3'], 
                   polarization='VV')
aoi = gpd.read_file(aoi_path)

In [12]:
df_sel = df_geo[df_geo.intersects(aoi.geometry[0])]
unique_subswaths = df_sel['subswath'].unique()

In [20]:
wfl = Workflow(graph_path)
wfl['Read'].parameters['file'] = infiles[0]
wfl['Read(2)'].parameters['file'] = infiles[1]
for subswath in unique_subswaths[:]:
    print(f"Subswath {subswath}, bursts {df_sel[df_sel['subswath']==subswath]['burst'].values}")
    wfl['TOPSAR-Split'].parameters['subswath'] = subswath
    wfl['TOPSAR-Split(2)'].parameters['subswath'] = subswath
    bursts = df_sel[df_sel['subswath']==subswath]['burst'].values
    burst_min = bursts.min()
    burst_max = bursts.max()
    wfl['TOPSAR-Split'].parameters['firstBurstIndex'] = burst_min
    wfl['TOPSAR-Split'].parameters['lastBurstIndex'] = burst_max
    wfl['TOPSAR-Split(2)'].parameters['firstBurstIndex'] = burst_min
    wfl['TOPSAR-Split(2)'].parameters['lastBurstIndex'] = burst_max
    # wfl['Write'].parameters['file'] = f"{outdir}/{subswath}_IFG.tif"
    wfl['Write'].parameters['file'] = f"{outdir}/{subswath}_COH_geo.tif"
    wfl.write('/tmp/graph.xml')
    grp = groupbyWorkers('/tmp/graph.xml', n=1)
    # gpt('/tmp/graph.xml', groups=grp, tmpdir='/data/tmp/')

    # remove bands
    with rio.open(f"{outdir}/{subswath}_COH_geo.tif", 'r') as src:
        prof = src.profile.copy()
        print(prof)
        prof.update({"driver": "GTiff",
                "nodata": 0
                 })
        struct = np.ones((15,15))
        with rio.open(f"{outdir}/{subswath}_COH_geo_border.tif", 'w', **prof) as dst:
            for i in range(1, prof['count'] + 1):
                band_src = src.read(i)
                msk_src = band_src != 0
                msk_dst = binary_erosion(msk_src, struct)
                band_dst = band_src * msk_dst
                dst.write(band_dst, i)

Subswath IW2, bursts [4 5 6]
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 6132, 'height': 4967, 'count': 1, 'crs': CRS.from_epsg(32633), 'transform': Affine(20.0, 0.0, -933398.0840116986,
       0.0, -20.0, 5543892.955377446), 'blockxsize': 288, 'blockysize': 560, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
Subswath IW3, bursts [4 5 6 7]
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 5782, 'height': 5534, 'count': 1, 'crs': CRS.from_epsg(32633), 'transform': Affine(20.0, 0.0, -845805.6712051213,
       0.0, -20.0, 5540669.063751001), 'blockxsize': 272, 'blockysize': 624, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}


In [ ]:

# with rio.open(f"{outdir}/{subswath}_IFG.tif") as ds:
# with rio.open(f"{outdir}/{subswath}_COH.tif") as ds:
with rio.open(f"{outdir}/{subswath}_COH_geo.tif") as ds:
    print(ds.profile)
    img = ds.read(1)
    # img = ds.read(3)
    # re = ds.read(1)
    # im = ds.read(2)


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.imshow(img[::8,::8], interpolation='nearest')#, vmin=0, vmax=100000)
plt.colorbar()

In [58]:
ds_iw2 = rio.open(f"{outdir}/IW2_COH_geo.tif")
ds_iw3 = rio.open(f"{outdir}/IW3_COH_geo.tif")
arr_merge, trans_merge = merge.merge([ds_iw2, ds_iw3])
with rio.open(f"{outdir}/IW2_COH_geo.tif") as src:
        out_meta = src.meta.copy()    
out_meta.update({"driver": "GTiff",
                 "height": arr_merge.shape[1],
                 "width": arr_merge.shape[2],
                 "transform": trans_merge,
                 "nodata": 0
                 })
# with rio.open(f"{outdir}/mergedRasters.tif", "w", **out_meta) as dst:
#         dst.write(arr_merge)

# write as COG
with MemoryFile() as memfile:
    with memfile.open(**out_meta) as mem:
        # Populate the input file with numpy array
        mem.write(arr_merge)

        dst_profile = cog_profiles.get("deflate")
        cog_translate(
            mem,
            f"{outdir}/mergedRasters.tif",
            dst_profile,
            in_memory=True,
            quiet=True,
        )

/tmp/ipykernel_168195/3130524809.py:22: RasterioDeprecationWarning: Source dataset should be opened in read-only mode. Use of datasets opened in modes other than 'r' will be disallowed in a future version.
  cog_translate(


In [24]:
out_meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': 0,
 'width': 10162,
 'height': 5695,
 'count': 1,
 'crs': CRS.from_epsg(32633),
 'transform': Affine(20.0, 0.0, -933398.0840116986,
        0.0, -20.0, 5543892.955377446)}

In [25]:
ds_iw2 = rio.open(f"{outdir}/IW2_COH_geo_border.tif")
ds_iw3 = rio.open(f"{outdir}/IW3_COH_geo_border.tif")
arr_merge, trans_merge = merge.merge([ds_iw2, ds_iw3])
with rio.open(f"{outdir}/IW2_COH_geo_border.tif") as src:
        out_meta = src.meta.copy()    
out_meta.update({
                 "height": arr_merge.shape[1],
                 "width": arr_merge.shape[2],
                 "transform": trans_merge,
                 "nodata": 0
                 })
# with rio.open(f"{outdir}/mergedRasters.tif", "w", **out_meta) as dst:
#         dst.write(arr_merge)

# write as COG
with MemoryFile() as memfile:
    with memfile.open(**out_meta) as mem:
        # Populate the input file with numpy array
        mem.write(arr_merge)

        dst_profile = cog_profiles.get("deflate")
        cog_translate(
            mem,
            f"{outdir}/mergedRasters_border.tif",
            dst_profile,
            in_memory=True,
            quiet=True,
        )

/tmp/ipykernel_217093/1684583032.py:22: RasterioDeprecationWarning: Source dataset should be opened in read-only mode. Use of datasets opened in modes other than 'r' will be disallowed in a future version.
  cog_translate(


In [ ]:
plt.imshow(ds_merged[0][0])